# 環境構築

In [ ]:
# Clone the LeRobot repository from GitHub using git
!git clone https://github.com/huggingface/lerobot.git

Cloning into 'lerobot'...
remote: Enumerating objects: 44845, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 44845 (delta 42), reused 37 (delta 37), pack-reused 44794 (from 2)
Receiving objects: 100% (44845/44845), 220.90 MiB | 15.80 MiB/s, done.
Resolving deltas: 100% (29004/29004), done.
Filtering content: 100% (45/45), 69.03 MiB | 20.70 MiB/s, done.


In [ ]:
# Change the current working directory to the 'lerobot' folder
%cd lerobot

/content/lerobot


In [ ]:
# Install the current package along with the optional 'pusht' dependencies
!pip install ".[pusht,libero]"

Processing /content/lerobot
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 121.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.7/217.7 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.5/217.5 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting req

In [ ]:
!pip install num2words

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 16.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=49a0c081863bbc246febf1f65d33d1e62db4226d66fc57a6d045b713c96ecb12
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


# ACTを学習してみる

まず，講演前半でお話したEnd to Endの動作モデルで登場したACTをLeRobotのフレームワークで学習をしてみましょう．

## pushtタスクのデータセットで学習

In [ ]:
!lerobot-train \
    --output_dir=outputs/train/act_pusht \
    --policy.type=act \
    --dataset.repo_id=lerobot/pusht \
    --seed=100000 \
    --env.type=pusht \
    --env.episode_length=50 \
    --batch_size=64 \
    --steps=500 \
    --eval_freq=500 \
    --save_freq=500 \
    --wandb.enable=false \
    --policy.push_to_hub=false

2025-12-11 17:45:10.637418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765475110.657347   14358 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765475110.663285   14358 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765475110.678706   14358 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765475110.678731   14358 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765475110.678737   14358 computation_placer.cc:177] computation placer alr

## 学習したACTを評価

In [ ]:
!lerobot-eval \
    --policy.path=outputs/train/act_pusht/checkpoints/000500/pretrained_model \
    --env.type=pusht \
    --env.episode_length=300 \
    --eval.batch_size=10 \
    --eval.n_episodes=10 \
    --policy.use_amp=false \
    --policy.device=cuda

2025-12-11 18:10:04.753779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765476604.774018   21965 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765476604.780185   21965 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765476604.796081   21965 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765476604.796107   21965 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765476604.796111   21965 computation_placer.cc:177] computation placer alr

## Visualization

In [ ]:
from IPython.display import Video

input_video_path = input("Please input video path: ")

Video(input_video_path, embed=True, width=320, height=240)

Please input video path: /content/lerobot/outputs/eval/2025-12-11/18-10-14_pusht_act/videos/pusht_0/eval_episode_5.mp4


# Huggingfaceで公開されている学習済みのACTを利用する

## 学習済みACTを評価

In [ ]:
# https://huggingface.co/aadarshram/act_pusht
# ACT trained on PushT dataset for 80,000 training steps. Trained for 3.5 hrs on T4 GPU on Colab free.
!lerobot-eval \
    --policy.path=aadarshram/act_pusht \
    --env.type=pusht \
    --env.episode_length=300 \
    --eval.batch_size=10 \
    --eval.n_episodes=10 \
    --policy.use_amp=false \
    --policy.device=cuda

2025-12-11 18:04:30.166194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765476270.186591   20189 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765476270.192792   20189 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765476270.208345   20189 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765476270.208373   20189 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765476270.208377   20189 computation_placer.cc:177] computation placer alr

## Visualization

In [ ]:
from IPython.display import Video

input_video_path = input("Please input video path: ")

Video(input_video_path, embed=True, width=320, height=240)

Please input video path: /content/lerobot/outputs/eval/2025-12-11/18-04-41_pusht_act/videos/pusht_0/eval_episode_6.mp4


# Diffusion policyを学習してみる

LeRobotフレームワークのいいところは公式実装のモデルはすぐに引数を変えるだけで学習，評価できてしまうところです．

## pushtタスクのデータセットで学習

In [ ]:
!lerobot-train \
    --output_dir=outputs/train/diffusion_pusht \
    --policy.type=diffusion \
    --dataset.repo_id=lerobot/pusht \
    --seed=100000 \
    --env.type=pusht \
    --env.episode_length=50 \
    --batch_size=64 \
    --steps=500 \
    --eval_freq=500 \
    --save_freq=500 \
    --wandb.enable=false \
    --policy.push_to_hub=false


2025-12-11 17:11:50.395339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765473110.416349    5361 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765473110.422803    5361 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765473110.439053    5361 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765473110.439081    5361 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765473110.439084    5361 computation_placer.cc:177] computation placer alr

## 学習したDiffusion policyを評価

In [ ]:
!lerobot-eval \
    --policy.path=outputs/train/diffusion_pusht/checkpoints/000500/pretrained_model \
    --env.type=pusht \
    --env.episode_length=300 \
    --eval.batch_size=10 \
    --eval.n_episodes=10 \
    --policy.use_amp=false \
    --policy.device=cuda


2025-12-11 18:16:31.961830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765476991.982617   23938 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765476991.988862   23938 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765476992.005425   23938 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765476992.005452   23938 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765476992.005456   23938 computation_placer.cc:177] computation placer alr

## Visualization

In [ ]:
from IPython.display import Video

input_video_path = input("Please input video path: ")

Video(input_video_path, embed=True, width=320, height=240)

Please input video path: /content/lerobot/outputs/eval/2025-12-11/18-16-42_pusht_diffusion/videos/pusht_0/eval_episode_2.mp4


# Huggingfaceで公開されている学習済みDiffusion policyを利用する

## 学習済みのDiffusion policyを評価
(公式version)

In [ ]:
!lerobot-eval \
    --policy.path=lerobot/diffusion_pusht \
    --env.type=pusht \
    --env.episode_length=300 \
    --eval.batch_size=10 \
    --eval.n_episodes=10 \
    --policy.use_amp=false \
    --policy.device=cuda

2025-12-11 18:24:45.216339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765477485.247934   26351 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765477485.257978   26351 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765477485.286847   26351 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765477485.286882   26351 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765477485.286891   26351 computation_placer.cc:177] computation placer alr

(ちゃんと動くものを公開してくれているversion)

In [ ]:
# https://huggingface.co/kimho/diffusion_pushT_ori
# 詳細不明

!lerobot-eval \
    --policy.path=kimho/diffusion_pushT_ori \
    --env.type=pusht \
    --env.episode_length=300 \
    --eval.batch_size=10 \
    --eval.n_episodes=10 \
    --policy.use_amp=false \
    --policy.device=cuda

Traceback (most recent call last):
  File "/usr/local/bin/lerobot-eval", line 5, in <module>
    from lerobot.scripts.lerobot_eval import main
  File "/usr/local/lib/python3.12/dist-packages/lerobot/scripts/lerobot_eval.py", line 73, in <module>
  File "/usr/local/lib/python3.12/dist-packages/lerobot/configs/eval.py", line 20, in <module>
    from lerobot import envs, policies  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lerobot/policies/__init__.py", line 17, in <module>
    from .groot.configuration_groot import GrootConfig as GrootConfig
  File "/usr/local/lib/python3.12/dist-packages/lerobot/policies/groot/__init__.py", line 18, in <module>
    from .modeling_groot import GrootPolicy
  File "/usr/local/lib/python3.12/dist-packages/lerobot/policies/groot/modeling_groot.py", line 42, in <module>
    from lerobot.policies.groot.groot_n1 import GR00TN15
  File "/usr/local/lib/python3.12/dist-packages/lerobot/policies/groot/groot_n

## Visualization

In [ ]:
from IPython.display import Video

input_video_path = input("Please input video path: ")

Video(input_video_path, embed=True, width=320, height=240)

Please input video path: /content/lerobot/outputs/eval/2025-12-11/18-34-43_pusht_diffusion/videos/pusht_0/eval_episode_2.mp4


# LIBERO simulatorを使ったVLAの評価
LIBEROはVLAのベンチマークとして最も有名なシミュレータの一つです．

最後にこのLIBEROのsimulatorを使ってColabでもギリギリ評価できるSmolVLAというVLAを評価してみましょう．

GPUの無料枠のbudgetを使い切って，GPUが一定期間使えなくなる可能性もあるので注意してください．


In [8]:
# https://huggingface.co/eunyoung927/smolvla-libero-smol

!export MUJOCO_GL=egl
!export TOKENIZERS_PARALLELISM=false
!lerobot-eval \
    --policy.path=eunyoung927/smolvla-libero-smol \
    --env.type=libero \
    --env.task=libero_object \
    --eval.batch_size=1 \
    --eval.n_episodes=1 \
    --policy.n_action_steps=1 \
    --env.max_parallel_tasks=1

Streaming output truncated to the last 5000 lines.
Running rollout with at most 280 steps:  20% 57/280 [00:41<02:30,  1.48it/s, running_success_rate=0.0%]
Running rollout with at most 280 steps:  20% 57/280 [00:41<02:30,  1.48it/s, running_success_rate=0.0%]
Running rollout with at most 280 steps:  21% 58/280 [00:41<02:31,  1.47it/s, running_success_rate=0.0%]
Running rollout with at most 280 steps:  21% 58/280 [00:42<02:31,  1.47it/s, running_success_rate=0.0%]
Running rollout with at most 280 steps:  21% 59/280 [00:42<02:30,  1.47it/s, running_success_rate=0.0%]
Running rollout with at most 280 steps:  21% 59/280 [00:43<02:30,  1.47it/s, running_success_rate=0.0%]
Running rollout with at most 280 steps:  21% 60/280 [00:43<02:29,  1.47it/s, running_success_rate=0.0%]
Running rollout with at most 280 steps:  21% 60/280 [00:43<02:29,  1.47it/s, running_success_rate=0.0%]
Running rollout with at most 280 steps:  22% 61/280 [00:43<02:27,  1.49it/s, running_success_rate=0.0%]
Running rollo

In [9]:
from IPython.display import Video

input_video_path = input("Please input video path: ")

Video(input_video_path, embed=True, width=320, height=240)

Please input video path: /content/lerobot/outputs/eval/2025-12-11/20-15-57_libero_smolvla/videos/libero_object_0/eval_episode_0.mp4
